<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [50]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')

**texto en negrita**

### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [51]:
# FIXME

df = pd.DataFrame()

#### a)

In [52]:
# Consolidamos la información de los archivos de años.

df_anio_list = []
for archivo in archivos_anio:
    df_temp = pd.read_csv(archivo)
    # Normalizar nombres de columnas a minúscula
    df_temp.columns = df_temp.columns.str.lower()
    df_anio_list.append(df_temp)

df_anio = pd.concat(df_anio_list, ignore_index=True)
display(df_anio.head())

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN


#### b)

In [53]:
# Se explora y limpiar df_codigos.

# Identificar el código ISO asociado a dos nombres de país distintos
codigo_duplicado = df_codigos[df_codigos.duplicated(subset=['codigo_iso'], keep=False)]
display("Código ISO duplicado:", codigo_duplicado)

# Eliminar el registro incorrecto ('malo')
df_codigos_cleaned = df_codigos[df_codigos['pais'].str.lower() != 'malo'].copy()
display("df_codigos después de la limpieza:", df_codigos_cleaned.head())

'Código ISO duplicado:'

,codigo_iso,pais
179,ZWE,Zimbabue
180,ZWE,malo


'df_codigos después de la limpieza:'

,codigo_iso,pais
0,AFG,Afghanistán
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,Emiratos Árabes Unidos


#### c)

In [54]:
# Se combina df_anio con df_codigos.
df = pd.merge(df_anio, df_codigos_cleaned, on='codigo_iso', how='inner')
display(df.head())

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

#### Estructura del DataFrame

In [55]:
# Analizamos la estructura del DataFrame.
print("Número de filas:", df.shape[0])
print("Número de columnas:", df.shape[1])
print("\nNombres de las columnas:", df.columns.tolist())
print("\nTipos de datos de cada columna:")
df.info()
print("\n Las columnas que se podrían considerar como con tipos de datos incoherentes son: codigo_iso y pais. Pero igual es esperado")

Número de filas: 3060
Número de columnas: 5

Nombres de las columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos de cada columna:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB

 Las columnas que se podrían considerar como con tipos de datos incoherentes son: codigo_iso y pais. Pero igual es esperado


#### Resumen estadístico

In [56]:
# Se realiza un resumen estadístico.
print("Resumen estadístico del DataFrame:")
print(df.describe())
print("El indice varía de un valor muy bajo (más libertad) a uno bastante alto (menos libertad), y su promedio queda en un punto intermedio. El ranking, en cambio, va del 1 (mejor posición) hasta el total de países evaluados. En general, un índice bajo coincide con un buen ranking, lo que muestra que ambas variables están alineadas.")

print("\nValores mínimo, máximo y promedio del índice:")
print("Mínimo:", df['indice'].min())
print("Máximo:", df['indice'].max())
print("Promedio:", df['indice'].mean())

# Países con valores extremos en indice y ranking
pais_min_indice = df.loc[df['indice'].idxmin(), ['pais', 'indice']]
pais_max_indice = df.loc[df['indice'].idxmax(), ['pais', 'indice']]
pais_min_ranking = df.loc[df['ranking'].idxmin(), ['pais', 'ranking']]
pais_max_ranking = df.loc[df['ranking'].idxmax(), ['pais', 'ranking']]

print("\nPaís con el menor índice (mayor libertad):")
print(pais_min_indice)
print("\nPaís con el mayor índice (menor libertad):")
print(pais_max_indice)
print("\nPaís con el menor ranking (mayor libertad):")
print(pais_min_ranking)
print("\nPaís con el mayor ranking (menor libertad):")
print(pais_max_ranking)


Resumen estadístico del DataFrame:
              anio        indice        ranking
count  3060.000000   2664.000000    2837.000000
mean   2009.941176    205.782316     477.930913
std       5.786024   2695.525264    6474.935347
min    2001.000000      0.000000       1.000000
25%    2005.000000     15.295000      34.000000
50%    2009.000000     28.000000      70.000000
75%    2015.000000     41.227500     110.000000
max    2019.000000  64536.000000  121056.000000
El indice varía de un valor muy bajo (más libertad) a uno bastante alto (menos libertad), y su promedio queda en un punto intermedio. El ranking, en cambio, va del 1 (mejor posición) hasta el total de países evaluados. En general, un índice bajo coincide con un buen ranking, lo que muestra que ambas variables están alineadas.

Valores mínimo, máximo y promedio del índice:
Mínimo: 0.0
Máximo: 64536.0
Promedio: 205.7823160660661

País con el menor índice (mayor libertad):
pais      Dinamarca
indice          0.0
Name: 1304, dtype:

#### Datos faltantes

In [57]:
# Se analiza los datos faltantes.
print("Número de valores nulos por columna:")
display(df.isnull().sum())

print("\nProporción de valores faltantes:")
display(df.isnull().sum() / len(df) * 100)

print("\nColumnas con más del 30% de datos faltantes:")
missing_percentage = df.isnull().sum() / len(df) * 100
display(missing_percentage[missing_percentage > 30])

Número de valores nulos por columna:


,0
codigo_iso,0
anio,0
indice,396
ranking,223
pais,0



Proporción de valores faltantes:


,0
codigo_iso,0.000000
anio,0.000000
indice,12.941176
ranking,7.287582
pais,0.000000



Columnas con más del 30% de datos faltantes:


,0


#### Unicidad y duplicados

In [58]:
# Se analiza y observa la unicidad de los datos, además de duplicación de ellos.
print("Número de países distintos:", df['pais'].nunique())
print("Número de años distintos:", df['anio'].nunique())
print("Número de filas duplicadas:", df.duplicated().sum())

Número de países distintos: 179
Número de años distintos: 17
Número de filas duplicadas: 0


#### Validación cruzada de columnas

In [59]:
# Se observa la validación cruzada de columnas
print("Validación cruzada de país y código ISO:")
display(df.groupby('codigo_iso')['pais'].nunique())

Validación cruzada de país y código ISO:


,pais
codigo_iso,
AFG,1
AGO,1
ALB,1
AND,1
ARE,1
...,...
WSM,1
YEM,1
ZAF,1





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [60]:
# Filter the DataFrame for Latin American countries
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)].copy()
display(df_america.head())

,codigo_iso,anio,indice,ranking,pais
5,ARG,2001,12.0,8.0,Argentina
7,ATG,2001,NaN,NaN,Antigua y Barbuda
20,BLZ,2001,NaN,NaN,Belize
21,BOL,2001,14.5,13.0,Bolivia
22,BRA,2001,18.8,18.0,Brasil


#### a)

In [61]:
# a) Se encuentran países con índice mínimo/máximo por año usando un ciclo for.
print("Países con índice mínimo y máximo por año (usando un ciclo):")

for anio in df_america['anio'].unique():
    df_anio = df_america[df_america['anio'] == anio].dropna(subset=['indice'])

    if not df_anio.empty:
        min_indice_pais = df_anio.loc[df_anio['indice'].idxmin()]
        max_indice_pais = df_anio.loc[df_anio['indice'].idxmax()]

        print(f"\nAño: {anio}")
        print("  Menor índice (mayor libertad):")
        display(min_indice_pais[['pais', 'indice']])

        print("  Mayor índice (menor libertad):")
        display(max_indice_pais[['pais', 'indice']])
    else:
        print(f"\nAño: {anio} - No existen datos válidos en 'indice'.")


Países con índice mínimo y máximo por año (usando un ciclo):

Año: 2001
  Menor índice (mayor libertad):


,27
pais,Canadá
indice,0.8


  Mayor índice (menor libertad):


,39
pais,Cuba
indice,90.3



Año: 2002
  Menor índice (mayor libertad):


...

,2739
pais,Cuba
indice,68.9



Año: 2019
  Menor índice (mayor libertad):


,2961
pais,Jamaica
indice,11.13


  Mayor índice (menor libertad):


,2919
pais,Cuba
indice,63.81


#### b)

In [62]:
# b) Se encuentran a los países con índice mínimo y máximo por año usando groupby.
print("\nPaíses con índice mínimo y máximo por año (usando groupby):")
df_america_limpio = df_america.dropna(subset=['indice'])

min_indice_groupby = df_america_limpio.loc[
    df_america_limpio.groupby('anio')['indice'].idxmin()
]

max_indice_groupby = df_america_limpio.loc[
    df_america_limpio.groupby('anio')['indice'].idxmax()
]

print("Menor índice (mayor libertad):")
display(min_indice_groupby[['anio', 'pais', 'indice']])

print("\nMayor índice (menor libertad):")
display(max_indice_groupby[['anio', 'pais', 'indice']])



Países con índice mínimo y máximo por año (usando groupby):
Menor índice (mayor libertad):


,anio,pais,indice
27,2001,Canadá,0.80
343,2002,Trinidad y Tobago,1.00
523,2003,Trinidad y Tobago,2.00
703,2004,Trinidad y Tobago,2.00
741,2005,Bolivia,4.50
927,2006,Canadá,4.88
1107,2007,Canadá,3.33
1287,2008,Canadá,3.70
1611,2009,Estados Unidos,6.75
1701,2012,Jamaica,9.88



Mayor índice (menor libertad):


,anio,pais,indice
39,2001,Cuba,90.30
219,2002,Cuba,97.83
365,2003,Argentina,35826.00
579,2004,Cuba,87.00
759,2005,Cuba,95.00
939,2006,Cuba,96.17
1119,2007,Cuba,88.33
1299,2008,Cuba,94.00
1479,2009,Cuba,78.00
1659,2012,Cuba,71.64


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [63]:
# Se crear la tabla dinámica (pivot_table) pedida en tarea principal usando el hint.
tabla_dinamica_indice = pd.pivot_table(
    df,
    values='indice',
    index='pais',
    columns='anio',
    aggfunc='max',
    fill_value=0
)

display(tabla_dinamica_indice.head())

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63


#### a)

In [64]:
# Se entrega el país con el mayor y menor (distinto de cero) índice en toda la tabla resultante.
max_indice_total = tabla_dinamica_indice.max().max()
min_indice_total = tabla_dinamica_indice[tabla_dinamica_indice > 0].min().min()  # excluir ceros
pais_max_total = tabla_dinamica_indice[tabla_dinamica_indice == max_indice_total].stack().idxmax()[0]
pais_min_total = tabla_dinamica_indice[tabla_dinamica_indice == min_indice_total].stack().idxmin()[0]

print(f"País con el mayor índice en toda la tabla: {pais_max_total} ({max_indice_total})")
print(f"País con el menor índice (distinto de cero) en toda la tabla: {pais_min_total} ({min_indice_total})")

País con el mayor índice en toda la tabla: Kosovo (64536.0)
País con el menor índice (distinto de cero) en toda la tabla: Austria (0.5)


#### b)

In [65]:
#  Se presentan los años con el promedio de índice más alto y más bajo.

promedio_indice_por_anio = tabla_dinamica_indice.mean(axis=0)
anio_mayor_promedio = promedio_indice_por_anio.idxmax()
anio_menor_promedio = promedio_indice_por_anio.idxmin()

print(f"Año con el promedio de índice más alto: {anio_mayor_promedio} ({promedio_indice_por_anio.max()})")
print(f"Año con el promedio de índice más bajo: {anio_menor_promedio} ({promedio_indice_por_anio.min()})")

Año con el promedio de índice más alto: 2013 (449.11446927374294)
Año con el promedio de índice más bajo: 2001 (20.03240223463687)


#### c)

In [66]:
# Se muestra el país con la mayor variabilidad en el índice a lo largo del tiempo.


variabilidad_indice = tabla_dinamica_indice.max(axis=1) - tabla_dinamica_indice.min(axis=1)

pais_mayor_variabilidad = variabilidad_indice.idxmax()
valor_max_variabilidad = variabilidad_indice.max()

print(f"País con la mayor variabilidad en el índice: {pais_mayor_variabilidad} ({valor_max_variabilidad})")

País con la mayor variabilidad en el índice: Kosovo (64536.0)


#### d)

In [67]:
# Se visualizan los países con un índice constante en todos los años registrados (excluyendo aquellos que solo tienen ceros).

paises_indice_constante = tabla_dinamica_indice[
    (tabla_dinamica_indice.max(axis=1) == tabla_dinamica_indice.min(axis=1)) &
    (tabla_dinamica_indice.min(axis=1) > 0)
]

print("Países con índice constante en todos los años registrados (excluyendo los que solo tienen ceros):")
display(paises_indice_constante)

Países con índice constante en todos los años registrados (excluyendo los que solo tienen ceros):


anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,


#### e)

In [68]:
# Se muestran los países que no tienen ningún dato (es decir,todos los valores iguales a 0) y una breve explicación de ello.

paises_todo_ceros = tabla_dinamica_indice[tabla_dinamica_indice.sum(axis=1) == 0]

print("Países sin datos (todos los valores iguales a 0):")
display(paises_todo_ceros)

print("\nExplicación:")
print("Estos países no tenían datos registrados en los archivos originales ""('libertad_prensa_01.csv' y 'libertad_prensa_02.csv') para ninguno de los años ""incluidos en el dataset. Al momento de crear la tabla dinámica, el parámetro `fill_value=0` ""reemplazó los valores faltantes (NaN) con 0, lo que genera estas filas llenas de ceros.")

Países sin datos (todos los valores iguales a 0):


anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,



Explicación:
Estos países no tenían datos registrados en los archivos originales ('libertad_prensa_01.csv' y 'libertad_prensa_02.csv') para ninguno de los años incluidos en el dataset. Al momento de crear la tabla dinámica, el parámetro `fill_value=0` reemplazó los valores faltantes (NaN) con 0, lo que genera estas filas llenas de ceros.
